In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer

In [3]:
file_path='/home/chirag/Loan_approval_predictor/Data/loan.csv'
df=pd.read_csv(file_path)

/tmp/ipykernel_23309/3575998743.py:2: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [4]:
#Handling missing values
threshold=0.4# we remove any columns with more than 40% missing values
df=df.dropna(thresh=len(df)*(1-threshold),axis=1)

In [5]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-13,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0


In [6]:
#filling missing neumerical data with median
num_col=df.select_dtypes(include=[np.number]).columns
num_imputer=SimpleImputer(strategy='median')
df[num_col]=num_imputer.fit_transform(df[num_col])

In [7]:
#removing nan values
nan_columns = df.columns[df.isna().any()].tolist()
print(nan_columns)


['emp_title', 'emp_length', 'desc', 'title', 'revol_util', 'last_pymnt_d', 'last_credit_pull_d']


In [8]:
df['term'] = df['term'].str.extract(r'(\d+)').astype(float)  # Converting to float

df['emp_length'] = df['emp_length'].str.extract(r'(\d+)')  # Extracting numbers
df['emp_length'] = df['emp_length'].fillna(0).astype(float)  # Replacing NaN with 0 and converting to float


In [9]:
# Convert 'int_rate' and 'revol_util' to numeric by removing '%' and converting to float
df['int_rate'] = df['int_rate'].str.replace('%', '').astype(float)
df['revol_util'] = df['revol_util'].str.replace('%', '').astype(float)

In [10]:
# One-Hot Encoding categorical columns
categorical_cols = ['home_ownership', 'purpose', 'verification_status', 'application_type', 'grade', 'sub_grade', 'addr_state']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [11]:
#converting target variable to binary
df=df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]
df['loan_status']=df['loan_status'].map({'Fully Paid': 0, 'Charged Off': 1})

In [12]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_title,emp_length,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,1077501.0,1296599.0,5000.0,5000.0,4975.0,36.0,10.65,162.87,NaN,10.0,...,False,False,False,False,False,False,False,False,False,False
1,1077430.0,1314167.0,2500.0,2500.0,2500.0,60.0,15.27,59.83,Ryder,1.0,...,False,False,False,False,False,False,False,False,False,False
2,1077175.0,1313524.0,2400.0,2400.0,2400.0,36.0,15.96,84.33,NaN,10.0,...,False,False,False,False,False,False,False,False,False,False
3,1076863.0,1277178.0,10000.0,10000.0,10000.0,36.0,13.49,339.31,AIR RESOURCES BOARD,10.0,...,False,False,False,False,False,False,False,False,False,False
5,1075269.0,1311441.0,5000.0,5000.0,5000.0,36.0,7.90,156.46,Veolia Transportaton,3.0,...,False,False,False,False,False,False,False,False,False,False


In [13]:
#Feature selection
#dropping irrelevant data
cols_to_drop=['id', 'member_id', 'title', 'zip_code', 'emp_title', 'url', 'desc', 'issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']
df=df.drop(columns=[col for col in cols_to_drop if col in df.columns])

In [14]:
nan_columns = df.columns[df.isna().any()].tolist()
print(nan_columns)


['revol_util']


In [15]:
df['revol_util'] = df['revol_util'].fillna(df['revol_util'].median())



In [16]:
#Scaling nuemerical features
scaler = StandardScaler()
numeric_features = df.select_dtypes(include=['number']).columns.tolist()
numeric_features = [col for col in numeric_features if col != 'loan_status']

# Apply scaling
df[numeric_features] = scaler.fit_transform(df[numeric_features])


In [17]:
from sklearn.preprocessing import LabelEncoder

# Create a copy of the dataset to avoid modifying the original
df_encoded = df.copy()

# Identify categorical columns, excluding 'loan_status'
categorical_cols = df_encoded.select_dtypes(include=['object']).columns
categorical_cols = categorical_cols.drop('loan_status', errors='ignore')  # Exclude 'loan_status' if present

# Apply Label Encoding to all categorical columns except 'loan_status'
label_encoders = {}  # Store encoders in case you need to inverse transform later

for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le  # Save encoder for later use

# Convert 'loan_status' to binary (0 and 1) if it's categorical
df_encoded['loan_status'] = df['loan_status'] 
    


In [18]:
df_encoded.sample(10)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,annual_inc,loan_status,pymnt_plan,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
30452,1.762706,1.863969,1.919514,-0.570835,-0.255255,2.220051,-0.847027,-0.058831,0,0,...,False,False,False,False,False,False,False,False,False,False
36484,-0.142484,-0.110583,-0.763545,-0.570835,0.161944,0.058493,-0.267967,-0.541625,1,0,...,False,False,False,False,False,False,False,False,False,False
12227,0.129686,0.171496,0.253113,-0.570835,-0.255255,0.337159,-0.847027,0.221465,0,0,...,False,False,False,False,False,False,False,False,False,False
24136,-0.822910,-0.815781,-0.743665,-0.570835,-0.626400,-0.776548,-0.847027,-0.058831,0,0,...,False,False,False,False,False,False,False,False,False,False
51,1.980442,2.089632,2.154110,-0.570835,-0.547837,2.408465,1.179682,0.641909,0,0,...,False,False,False,False,False,False,False,False,False,False
4599,1.898791,2.005009,2.068672,-0.570835,0.205290,2.473985,0.311092,0.361613,0,0,...,False,False,False,False,False,False,False,False,False,False
23534,0.946196,0.136236,0.210394,-0.570835,-1.663981,0.162405,1.469212,0.096889,0,0,...,False,False,False,False,True,False,False,False,False,False
8267,-0.822910,-0.815781,-0.743665,-0.570835,-0.363619,-0.765620,-1.426086,-0.416986,0,0,...,False,False,False,False,False,False,False,False,False,False
3946,-1.299207,-1.309419,-1.242053,-0.570835,-0.547837,-1.313894,1.469212,-0.323554,0,0,...,False,False,True,False,False,False,False,False,False,False
7160,0.810111,0.876693,0.965097,-0.570835,-0.060201,1.149529,-0.267967,0.019029,0,0,...,False,False,False,False,False,False,False,False,False,False


In [22]:
# Assuming df is the final preprocessed dataframe before training
feature_names = df.columns.to_numpy()  # Extract column names as an array
np.save("/home/chirag/Loan_approval_predictor/models/training_features.npy", feature_names)  # Save it
print("Feature names saved successfully!")


Feature names saved successfully!


In [21]:
# Save processed dataset
df_encoded.to_csv("/home/chirag/Loan_approval_predictor/Data/processed_loan.csv", index=False)
print("Data Preprocessing Completed! Processed file saved.")


Data Preprocessing Completed! Processed file saved.
